# image transfer のチュートリアる
- https://www.youtube.com/watch?v=imX4kSKDY7s
- 動かない！！

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [13]:
VGG19_PATH = './models/vgg19-d01eb7cb.pth'
# model = models.vgg19(pretrained = False)
# model.load_state_dict(torch.load(VGG19_PATH), strict=False)
model = models.vgg19(pretrained = True).features

/Users/kunia/opt/anaconda3/envs/practice2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kunia/opt/anaconda3/envs/practice2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/kunia/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [16]:
# def select_device():
#     # “”"GPU もしくは CPU の選択“”"
#     if torch.cuda.is_available():
#         device = torch.device('cuda')
#         print('cuda is selected as device!')
#     elif torch.backends.mps.is_available():
#         device = torch.device('mps')
#         print('mps is selected as device!')
#     else:
#         device = torch.device('cpu')
#         print('cpu....f')
#     return device
# device = select_device()

In [19]:
class VGG(nn.Module):
    def __init__(self) -> None:
        super(VGG, self).__init__()

        self.chosen_features = ['0', '5', '10', '19', '28']
        # model = models.vgg19(pretrained=False)
        # model.load_state_dict(torch.load(VGG19_PATH))
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []
        
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)

        return features

def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

device = torch.device('mps')
image_size = 356

loader = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ]
)
original_img = load_image('./cat.png')
style_img = load_image('./style_0.png')

model = VGG().to(device).eval()
generated = torch.randn(original_img.shape, device=device, requires_grad = True)
# generated = original_img.clone().requires_grad_(True)

total_steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01

optimizer = optim.Adam([generated], lr=learning_rate)

for step in range(total_steps):
    generated_features = model(generated)

    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(generated_features, original_img_features, style_features):
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

        G = gen_feature.views(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )

        A = style_feature.views(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )

        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta + style_loss
    optimizer.zero_grad()
    total_loss.backward()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, 'generated.png')





/Users/kunia/opt/anaconda3/envs/practice2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kunia/opt/anaconda3/envs/practice2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 4, 356, 356] to have 3 channels, but got 4 channels instead